In [2]:
import os
import pandas as pd
import sys
import pathlib
from IPython.display import Markdown
import xmltodict
from ipywidgets import widgets
from dataclasses import dataclass, asdict, field
from ipyaggrid import Grid

from _autoui import EditListOfDicts
%run _utils.py

# +
class PdtLayouts():
    @property
    def box_layout_split(self):
        return widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='50%')
    @property
    def box_layout_hide(self):
        return widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='5%')
    
    @property
    def box_layout_expand(self):
        return widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='95%')
    
    @property
    def box_layout_full(self):
        return widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='100%')

def get_di_default_types(fpths):
    """
    reads the data from all the CIBSE pdt xml fpths given and
    creates a list of all unique unit types:
    json_pdt['pdt']['params']['parameter']['units']['@type']
    and gives a default value to match the type (so the app 
    interprets numbers as numbers, text as text etc.)
    """
    def get_unit_types(fpth):
        json_pdt = xml_to_json(fpth,display_xml=False)
        params = json_pdt['pdt']['params']['parameter']
        units = [param['units'] for param in params]
        unit_types = [unit['@type'] for unit in units]
        return list(set(unit_types))
    def get_all_unit_types(fpths):
        all_unit_types = []
        for fpth in fpths:
            all_unit_types = all_unit_types + get_unit_types(fpth)
        return list(set(all_unit_types))
    unit_types = get_all_unit_types(fpths)
    di_default_types = {unit:0 for unit in unit_types}
    di_default_types['Text'] = 'Text'
    di_default_types['URL'] = 'URL'
    return di_default_types

def get_description(param):
    unittype = get_unittype(param)
    des = param['description']
    if des is None:
        des = unittype
    else:
        des = unittype + ', ' + des
    return des
        
def get_unittype(param):
    return param['units']['@type']

def get_di_ui(param,di_default_types):
    name = param['name']['#text']
    di_ui = {
        'name':name,
        'label':get_description(param)
    }
    value, options, dtype = get_values_options(param,di_default_types=di_default_types)
    di_ui['value'] = value
    if options is not None:
        di_ui['options'] = options
    return di_ui

def get_lidi_ui(params,di_default_types):
    lidi_ui = []
    for param in params:
        di_ui = get_di_ui(param,di_default_types)
        lidi_ui.append(di_ui)
    return lidi_ui


# -- 
def get_options(param):
    try:
        options = param['value']['enumItem']
    except:
        options = None
    return options
        
def get_values_options(param,di_default_types=None):
    options = get_options(param)
    if options is not None:
        value = options[0]
    else:
        if di_default_types is not None:
            value = di_default_types[get_unittype(param)]
            
        else:
            value = param['value']
    dtype = str(type(value))
    return value, options, dtype

def unpack_param(param,di_default_types=None):
    value, options, dtype = get_values_options(param,di_default_types=di_default_types)
    di = {
        'guid':[param['guid']['#text']],
        'guid_source':[param['guid']['@source']],
        'name':[param['name']['#text']],
        'idx':[param['name']['@idx']],
        'description':[param['description']],
        'notes':[param['notes']],
        'category':[param['category']],
        'section':[param['section']['#text']],
        'units_type':[param['units']['@type']],
        'options':[options],
        'value':[value],
        'dtype':[dtype]
            }
    return di

def get_df_params(params,di_default_types=None):
    df = df_from_list_of_dicts(params, list(params[0].keys()))
    li = []
    cols = ['index','guid','guid_source']
    df_params = pd.DataFrame(columns=cols)
    for index, row in df.iterrows():
        di = {'index':[index]}
        di.update(unpack_param(row,di_default_types=di_default_types))
        df_params = df_params.append(pd.DataFrame.from_dict(di))
    df_params = df_params.set_index('index',drop=True)
    df_params['description'] = df_params['description'].fillna('')
    df_params['label'] = df_params.units_type #+ ', ' + df_params.description
    return df_params
# --

@dataclass
class PdtLayouts():
    box_layout_split = widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='50%')
    
    box_layout_hide = widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='5%')
    
    box_layout_expand = widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='95%')
    
    box_layout_full = widgets.Layout(display='flex',
                            flex_flow='columns',
                            align_items='stretch',
                            border='solid',
                            width='100%')
    
       
class CibsePdtBase():  
    
    def __init__(self,fpth_xmlpdt, di_default_types):
        self.fpth_xmlpdt = fpth_xmlpdt
        self.di_default_types = di_default_types
        self._init_CibsePdtBase()
        
    def _init_CibsePdtBase(self):
        self.json_pdt = xml_to_json(self.fpth_xmlpdt,display_xml=False)
        self.df_params = self._get_df_params()
        
    @property
    def json_pdt(self): 
        return self._json_pdt 
    
    @json_pdt.setter
    def json_pdt(self, json_pdt):
        self._json_pdt = json_pdt
        
    @property
    def df_params(self): 
        return self._df_params
    
    @df_params.setter
    def df_params(self, df_params):
        self._df_params = df_params
        
    def _get_df_params(self):
        return get_df_params(self.params,di_default_types=self.di_default_types)
    
    @property
    def params(self):
        return self.json_pdt['pdt']['params']['parameter']
        
    @property
    def header(self):
        return self.json_pdt['pdt']['header']
    
    @property
    def header_md(self):
        #str_md = """"""
        #for k,v in self.json_pdt['pdt']['header'].items():
        #    str_md = str_md+'__{0}__: {1}<br>'.format(k,v)
            
        str_md = pd.DataFrame.from_dict(
            self.json_pdt['pdt']['header'],
            orient='index').rename(columns={0:'header'}
                                  ).to_markdown()
        return str_md
    
    @property
    def header_display(self):
        display(Markdown(self.header_md))
        
class CibsePdtUI(CibsePdtBase,PdtLayouts):
    
    def __init__(self,fpth_xmlpdt, di_default_types):
        self.fpth_xmlpdt = fpth_xmlpdt
        self.di_default_types = di_default_types
        self._init_CibsePdtUI()

    def _init_CibsePdtUI(self):
        self._init_CibsePdtBase()
        #self.get_ui_inputs()
        self._ui_data()
        self._buildui()
        self._init_controls()
        #self._init_form()
        
    @property
    def sections(self):    
        return self.df_params.section.unique()
    
    def _ui_data(self):    
        cols = ['name','value','label','options']
        self.ui_data = {}
        for s in self.sections:
            self.ui_data[s] = self.df_params.query('section == "{0}"'.format(s))[cols].to_dict('records')
        
    def _buildui(self):
        self.ui_bits = {}
        self.ui_section = {}
        for section,di_params in self.ui_data.items():
            button_showhide = widgets.ToggleButton(description='+', layout=widgets.Layout(width='30px'),tooltip='show / hide section parameters',style={'font_weight': 'bold','button_color':'white'})       
            title = widgets.HTML('<b>{0}</b>'.format(section))
            header = widgets.HBox([button_showhide,title])
            ui_params = EditListOfDicts(di_params).applayout
            ui_container = widgets.VBox([])
            ui_section = widgets.VBox([header,ui_container])
            self.ui_bits[section]={}
            self.ui_bits[section]['button_showhide'] = button_showhide
            self.ui_bits[section]['title'] = title
            self.ui_bits[section]['header'] = header
            self.ui_bits[section]['ui_params'] = ui_params
            self.ui_bits[section]['ui_container'] = ui_container
            self.ui_section[section] = ui_section
        self.pdtname = os.path.splitext(os.path.basename(self.fpth_xmlpdt))[0]
        self.title = widgets.HTML('<b>{0}</b>'.format(self.pdtname))
        children = [v for k,v in self.ui_section.items()] #[self.title] + 
        self.ui = widgets.VBox(children)    #, layout=self.box_layout_split
            
    def _init_controls(self):
        for section in self.sections:
            self.ui_bits[section]['button_showhide'].observe(self.showhide_section, names='value')
            
    def showhide_section(self,onchange):
        for section in self.sections:
            if self.ui_bits[section]['button_showhide'].value:
                self.ui_bits[section]['ui_container'].children = [self.ui_bits[section]['ui_params']]
                self.ui_bits[section]['button_showhide'].description = '-'
            else:
                self.ui_bits[section]['ui_container'].children = []
                self.ui_bits[section]['button_showhide'].description = '+'
            
    def display_CibsePdtUI(self):
        display(self.ui)   
            
    def _ipython_display_(self):
        self.display_CibsePdtUI()      


# +
def gen_column_defs(li,getkey='aggrid_column_defs',ignoreifnokey=False):
    """
    generates column definitions
    """
    def create_coldef_di(l,getkey):
        di = {'field':l['name'],'headerName':l['name'],'sortable':True}
        try:
            if 'editable' in l[getkey].keys():
                if 'options' in l.keys():
                    di.update({'cellEditor' : 'agSelectCellEditor'})
                    di.update({'cellEditorParams':{'values':l['options']}})
        except:
            pass
        try:
            di.update(l[getkey])
        except:
            pass
        return di
    li1 = []
    for l in li:
        if ignoreifnokey and getkey not in l.keys():
            pass
        else:
            di = create_coldef_di(l,getkey)
            li1.append(di)
    return li1

def default_grid_options(col_defs):
    """
    global column definitions for AG-Grid
    """
    grid_options = {
        'columnDefs': col_defs,
        'defaultColDef': {
            'flex': 1,
            'minWidth': 80,
            'resizable': True,
            },
        'rowSelection': 'multiple',
        'sortable':True,
        'filter': True,
        'resizable': True,
        'enableRangeSelection': True,
        'suppressRowClickSelection': True,
        'columns_fit':"auto",
        'enableCellTextSelection':True
    } 
    return grid_options

def default_init_grid(grid_data,
                      grid_options,
                      height=700,
                      quick_filter=True,
                      keep_multiindex=True,
                      theme='ag-theme-balham',
                      sync_on_edit=True,
                      export_mode='auto'
                     ):
    return Grid(
            grid_data=grid_data,
            grid_options=grid_options,
            height=height,
            quick_filter=quick_filter,
            keep_multiindex=keep_multiindex,
            theme=theme,
            #js_helpers_custom = HELPERS_CUSTOM,
            sync_on_edit=sync_on_edit,
            export_mode=export_mode
               )

def make_col_def(name):
    return  {
        'name':name,
        'label':name,
        'aggrid_column_defs':{'columnGroupShow': 'closed'}
    }

def make_gr_tmp(pdt_ui):
    df_tmp1 = pdt_ui.df_params.set_index(['section', 'name'])
    index = df_tmp1.index
    row = pdt_ui.df_params.set_index(['section', 'name']).value 
    df_data = pd.DataFrame(index = index)
    li_grilles = ['GR'+str(n) for n in range(0,10)]
    
    for l in li_grilles:
        df_data[l] = row
    return df_data.reset_index(), li_grilles

# +

fdir = pathlib.Path('data')
fpth_pdts = recursive_glob(rootdir=fdir,pattern='*.xml',recursive=False)
fpth_pdt = fpth_pdts[0]

fpth_pdts  = fpth_pdts + [fpth_pdt] #

json_pdt = xml_to_json(fpth_pdt,display_xml=True)
di_default_types = get_di_default_types(fpth_pdts)
fpth_default_types = pathlib.Path('data/default_types.json')
write_json(di_default_types,fpth=fpth_default_types)
pdt_ui = CibsePdtUI(fpth_pdt,di_default_types)
defs = [
    {
        'name':'section',
        'label':'section',
        'aggrid_column_defs':{'minWidth':150,
                              'type': 'textColumn',
                              'pinned':'left',
                              'headerCheckboxSelection': True,
                              'headerCheckboxSelectionFilteredOnly': True,
                              'checkboxSelection': True},
    },
    {
        'name':'name',
        'label':'name',
        'aggrid_column_defs':{'columnGroupShow': 'closed','pinned':'left'}
    },
]

type_mark = 'GR1'
cols = ['section','name','value']
#df_tmp = pdt_ui.df_params[cols].rename(columns={'value':type_mark})
df_tmp, li_grilles = make_gr_tmp(pdt_ui)
defs = defs + [make_col_def(name) for name in li_grilles]
defs = gen_column_defs(defs, getkey='aggrid_column_defs', ignoreifnokey=False)
grid_options = default_grid_options(defs)
gr_tmp = default_init_grid(df_tmp,grid_options)

# box left ---------------------
# header
hide = widgets.Button(icon='eye-slash',layout={'width':'40px'})
copydialogue = widgets.Button(icon='copy')
adddialogue = widgets.Button(icon='plus')
removedialogue = widgets.Button(icon='minus')
box_header = widgets.HBox([
    hide, 
    copydialogue,
    adddialogue,
    removedialogue])#,layout=pdt_ui.box_layout_split

# copy dialogue 
copy = widgets.Button(description='copy data', button_style='success')
copyfrom= widgets.Dropdown(description='copy from:', options=li_grilles)
copyto= widgets.SelectMultiple(description='copy to:', value=[], options=li_grilles)
params = widgets.Dropdown(description='parameters:', options=['all', 'selected rows'])
box_copy = widgets.HBox([
    copy,
    copyfrom,
    copyto,
    params
])

# add dialogue
add = widgets.Button(description='add new', button_style='success')
addfrom = widgets.Dropdown(description='copy from:', options=li_grilles)
box_add = widgets.HBox([
    add,
    addfrom,
])

# add dialogue
remove = widgets.Button(description='remove', button_style='danger')
removefrom = widgets.SelectMultiple(description='remove:', options=li_grilles)
box_remove = widgets.HBox([
    remove,
    removefrom,
])

# make right containers
box_data = widgets.VBox([gr_tmp])
title_left = f"All instances of {pdt_ui.pdtname}'s in project"
title_left = widgets.HTML(f'<b>{title_left}</b>')
box_left = widgets.VBox([title_left,box_header,box_data], layout=pdt_ui.box_layout_split)

# box left ---------------------
title_right = f"Editable instance of {pdt_ui.pdtname}"
title_right = widgets.HTML(f'<b>{title_right}</b>')
selectitem = widgets.Dropdown(description='edit item:', options=li_grilles)
saveitem = widgets.Button(description='save', icon='save', button_style='success')
box_edititem = widgets.HBox([selectitem, saveitem])
box_right = widgets.VBox([title_right, box_edititem, pdt_ui.ui],layout=pdt_ui.box_layout_split)

# box pdts ---------------------
box_pdts = widgets.HBox([box_left,box_right])

def copydialogue_onclick(click):
    box_data.children = [box_copy, gr_tmp]
    
def adddialogue_onclick(click):
    box_data.children = [box_add, gr_tmp]

def removedialogue_onclick(click):
    box_data.children = [box_remove, gr_tmp]
    
def hide_onclick(click):
    box_data.children = [gr_tmp]
        
copydialogue.on_click(copydialogue_onclick)
adddialogue.on_click(adddialogue_onclick)
removedialogue.on_click(removedialogue_onclick)
hide.on_click(hide_onclick)

tab = widgets.Tab()
tab.children = [gr_tmp,box_pdts]
tab.set_title(0,'template data')
tab.set_title(1,'project data')
tab.selected_index = 1

tab

/mnt/c/engDev/git_mf/pdtui/data/AirGrilleDiffuser.xml

<IPython.core.display.JSON object>